In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

from training_new import get_data_loaders, train
from model import Model, DummyModel
from inference import greedy_inference_single
from util import create_graph_no_sol

In [4]:
graphs_path = "./solver/sol_examples/vrp20/sols.pkl"

In [5]:
train_dl, valid_dl = get_data_loaders(graphs_path, batch_size=5, max_instances=20)

Parsing Graphs: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 105.26it/s]


In [12]:
model = Model(4, 20, 2, 0.0)
train(model, "./solver/sol_examples/vrp20/sols.pkl", epochs=5, batch_size=16, learning_rate=3e-3, max_instances=2000)

0.003


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2185/2185 [00:07<00:00, 294.25it/s]


Train 0 - loss:1.1918, acc:0.5872
Valid 0 - loss:1.0585, acc:0.6409


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2185/2185 [00:07<00:00, 284.91it/s]


Train 1 - loss:1.0033, acc:0.6596
Valid 1 - loss:1.0385, acc:0.6562


...

Train 2 - loss:0.9835, acc:0.6676
Valid 2 - loss:1.0111, acc:0.6747


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2185/2185 [00:06<00:00, 315.54it/s]


Train 3 - loss:0.9719, acc:0.6673
Valid 3 - loss:0.9853, acc:0.6801


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2185/2185 [00:06<00:00, 316.94it/s]


Train 4 - loss:0.9607, acc:0.6708
Valid 4 - loss:1.0037, acc:0.6746


In [21]:
graphs = pd.read_pickle(graphs_path)

In [22]:
graph_collections = [create_graph_no_sol(g) for g, _ in graphs]

In [37]:
solved_graphs = []
for g, orig_g in tqdm(list(zip(graph_collections, graphs))[:100]):
    g = deepcopy(graph_collections[0])
    g = deepcopy(g)
    g.update_node(orig_g[1][1][0]) # unf we have to do this for now
    res_g = greedy_inference_single(model, g)
    solved_graphs.append(res_g)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 27.41it/s]


In [38]:
solved_graphs[0].get_full_solution_cost()

6.862126483690885

In [39]:
np.mean([sg.get_full_solution_cost() for sg in solved_graphs])

8.23473345975078

In [73]:
from generator import *
from model import Model
from typing import List
from torch_geometric.data.batch import Batch

In [78]:
from copy import deepcopy

In [189]:
def greedy_inference_single(
    model: Model,
    graph: GraphCollection
)-> SolutionInstance:
    """Runs inference on a single graph.
    """
    # start from empty sol
    # TODO(roberto): What should we do here? start from 0 or not
    graph = deepcopy(graph)
    sol = []
    while not graph.is_complete():
        curr_graph = graph.get_current_unsolved_graph()
        curr_graph = curr_graph.export_pyg()
        curr_batch = Batch.from_data_list([curr_graph])
        next_probs = model.probs(curr_batch)
        print(next_probs)
        next_node = next_probs.squeeze().argmax().cpu().item()
        print(next_node)
        graph.update_node(next_node)
        sol.append(next_node)
        
    return graph

In [197]:
graphs = pd.read_pickle(graphs_path)

In [198]:
graph_collections = [create_graph_no_sol(g) for g, _ in graphs]

In [199]:
from model import Model, DummyModel

In [200]:
model = Model(4, 20, 2, 0.0)

In [201]:
g = graph_collections[0]

In [202]:
g.update_node(graphs[0][1][1][0])

In [203]:
res = greedy_inference_single(model, g)

tensor([[0.0671, 0.0433, 0.0450, 0.0422, 0.0462, 0.0538, 0.0445, 0.0682, 0.0480,
         0.0481, 0.0000, 0.0495, 0.0570, 0.0491, 0.0539, 0.0425, 0.0439, 0.0458,
         0.0445, 0.0457, 0.0616]], grad_fn=<SoftmaxBackward0>)
7
tensor([[0.0681, 0.0484, 0.0491, 0.0476, 0.0497, 0.0600, 0.0499, 0.0000, 0.0546,
         0.0527, 0.0000, 0.0513, 0.0567, 0.0530, 0.0541, 0.0460, 0.0474, 0.0501,
         0.0480, 0.0497, 0.0636]], grad_fn=<SoftmaxBackward0>)
0
tensor([[0.0000, 0.0503, 0.0519, 0.0493, 0.0534, 0.0636, 0.0525, 0.0000, 0.0571,
         0.0561, 0.0000, 0.0565, 0.0643, 0.0567, 0.0600, 0.0491, 0.0511, 0.0531,
         0.0515, 0.0526, 0.0707]], grad_fn=<SoftmaxBackward0>)
20
tensor([[0.0807, 0.0491, 0.0511, 0.0477, 0.0528, 0.0629, 0.0509, 0.0000, 0.0555,
         0.0551, 0.0000, 0.0568, 0.0665, 0.0561, 0.0608, 0.0482, 0.0501, 0.0529,
         0.0506, 0.0523, 0.0000]], grad_fn=<SoftmaxBackward0>)
0
tensor([[0.0000, 0.0542, 0.0560, 0.0531, 0.0576, 0.0684, 0.0566, 0.0000, 0.0615,
         0

In [164]:
res.get_full_solution_cost()

0, 0:  0.0
0, 0:  0.0
0, 10:  0.8097368074215897
10, 0:  0.8097368074215897
0, 14:  0.14358948746570221
14, 0:  0.14358948746570221
0, 12:  0.5285496926283793
12, 0:  0.5285496926283793
0, 5:  0.6359795993919065
5, 8:  0.39457206108319837
8, 13:  0.5498971197011205
13, 4:  0.37368186435237444
4, 0:  0.1634832648201612
0, 20:  0.4770585938744954
20, 0:  0.4770585938744954
0, 9:  0.8008364190621853
9, 0:  0.8008364190621853
0, 2:  0.28491531139383897
2, 0:  0.28491531139383897
0, 6:  0.9580574443402365
6, 0:  0.9580574443402365
0, 16:  0.3973094313399893
16, 0:  0.3973094313399893
0, 15:  0.17586047218358375
15, 0:  0.17586047218358375
0, 7:  0.8613240146413769
7, 0:  0.8613240146413769
0, 1:  0.614393979032243
1, 0:  0.614393979032243
0, 17:  0.8522194446501334
17, 0:  0.8522194446501334
0, 19:  0.6337251932269227
19, 0:  0.6337251932269227
0, 11:  0.6010141060953877
11, 0:  0.6010141060953877
0, 18:  0.717048167540401
18, 0:  0.717048167540401
0, 3:  0.6469427054570099
3, 0:  0.6469427

21.122776450055714

In [75]:
set([0, 1, 0])

{0, 1}

In [9]:
import pandas as pd

In [15]:
from util import *

In [57]:
graphs = pd.read_pickle(graphs_path)

In [58]:
graph_collections = [create_graph_no_sol(g) for g, _ in graphs]

In [74]:
type(g)

graph.GraphCollection

In [59]:
g = graph_collections[0]

In [60]:
g.visited_nodes

[0]

In [61]:
g.update_node(1)
g.update_node(2)

In [68]:
g.update_node(3)

In [69]:
g.visited_nodes

[0, 1, 2, 3]

In [70]:
x = g.get_current_unsolved_graph()

In [71]:
x.export_pyg()

Data(
  x=[21, 4],
  edge_index=[2, 3],
  edge_attr={},
  prev_node=3,
  vehicle_cap=17,
  y=10000
)

In [48]:
x = g.get_all_graphs()[0]

In [49]:
x.export_pyg()

Data(
  x=[21, 4],
  edge_index=[2, 1],
  edge_attr={},
  prev_node=1,
  vehicle_cap=25,
  y=2
)

In [50]:
x.get_edges()

EdgeView([(0, 1)])

In [56]:
g.curr_graph.export_pyg()

Data(
  x=[21, 4],
  edge_index=[2, 2],
  edge_attr={},
  prev_node=1,
  vehicle_cap=25,
  y=2
)

In [27]:
g.curr_graph.get_nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20))

In [29]:
curr_graph = g.curr_graph

In [33]:
g.get_all_graphs()

[]

In [32]:
curr_graph.get_edges()

EdgeView([(0, 1)])

In [30]:
curr_graph.export_pyg()

Data(
  x=[21, 4],
  edge_index=[2, 1],
  edge_attr={},
  prev_node=0,
  vehicle_cap=30,
  y=1
)

In [ ]:
g.

In [83]:
from model import Model, DummyModel

In [88]:
model = Model(3, 20, 2, 0.0)
train(model, "./solver/sol_examples/vrp20/sols.pkl", epochs=200, batch_size=16, learning_rate=3e-3, max_instances=2000)

0.003


Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2185/2185 [00:08<00:00, 251.50it/s]


Train 0 - loss:1.6576, acc:0.4253
Valid 0 - loss:1.3877, acc:0.5382


Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2185/2185 [00:08<00:00, 253.63it/s]


Train 1 - loss:1.3045, acc:0.5655
Valid 1 - loss:1.2983, acc:0.5662


...

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2185/2185 [00:08<00:00, 253.69it/s]


Train 68 - loss:1.1424, acc:0.6024
Valid 68 - loss:1.2101, acc:0.5775


Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2185/2185 [00:09<00:00, 235.92it/s]


Train 69 - loss:1.1416, acc:0.6016


KeyboardInterrupt: 

In [55]:
dm = DummyModel()
train(dm, "./solver/sol_examples/vrp20/sols.pkl", epochs=200, batch_size=16, learning_rate=3e-3, max_instances=500)

0.003


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 546/546 [00:01<00:00, 440.77it/s]


Train 0 - loss:1.7798, acc:0.4015
Valid 0 - loss:1.5175, acc:0.4862


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 546/546 [00:01<00:00, 452.86it/s]


Train 1 - loss:1.5231, acc:0.4885
Valid 1 - loss:1.4872, acc:0.4979


...

Train 196 - loss:1.2778, acc:0.5677
Valid 196 - loss:1.2891, acc:0.5651


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 546/546 [00:01<00:00, 435.20it/s]


Train 197 - loss:1.2775, acc:0.5685
Valid 197 - loss:1.2892, acc:0.5646


Batches:  54%|█████████████████████████████████████████████████████████████████▏                                                       | 294/546 [00:00<00:00, 404.69it/s]


KeyboardInterrupt: 